In [1]:
import os


In [2]:
%pwd

'c:\\Users\\user\\Desktop\\Insurance E2E ML\\Insurance-Premium-End-to-End-\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\user\\Desktop\\Insurance E2E ML\\Insurance-Premium-End-to-End-'

In [5]:
#entity

from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionCOnfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path
    comp_name: str

In [6]:
from src.inproject.constants import *
from src.inproject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_pathway = CONFIG_FILE_PATH,
                 param_pathway = PARAM_FILE_PATH,
                 schema_pathway = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_pathway)
        self.params = read_yaml(param_pathway)
        self.schema = read_yaml(schema_pathway)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionCOnfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionCOnfig(
            root_dir= config.root_dir,
            source_URL=config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir=config.unzip_dir,
            comp_name=config.comp_name
        )

        return data_ingestion_config

In [8]:
# data ingestion component

import os
import zipfile
import urllib.request as request
from src.inproject.logging import logger
from src.inproject.utils.common import read_yaml, get_size
from kaggle.api.kaggle_api_extended import KaggleApi
# import kaggle


In [9]:


class DataIngestion:
    def __init__(self, config=DataIngestionCOnfig):
        self.config = config

   
    def download_data_from_kaggle(self):
        if not os.path.exists(self.config.local_data_file):
            os.makedirs(self.config.root_dir, exist_ok=True)
            api = KaggleApi()
            api.authenticate()
            api.competition_download_files(self.config.comp_name, self.config.root_dir)

            logger.info(f'data downloaded succesfully ')

        else:
            logger.info(f'file already exist of size: {get_size(Path(self.config.local_data_file))}')


    def extract_zip_file(self):
        """
        zip_file_path: zip
        extract zip file into the directory
        function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
# pipeline

try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_data_from_kaggle()
    data_ingestion.extract_zip_file()


except Exception as e:
    raise e

[2025-01-02 02:17:49,854: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-02 02:17:49,856: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-02 02:17:49,859: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-02 02:17:49,861: INFO: common: created directory at: artifacts]
[2025-01-02 02:17:49,862: INFO: common: created directory at: artifacts/data_ingestion]
[2025-01-02 02:18:10,091: INFO: 2452134524: data downloaded succesfully ]
